# Classic Machine Learning

In [1]:
import numpy as np 

data = np.load("../../data/processed/sstReducedStateCOPERNICUS20102019Prepared.npz")

PCsTrain = data["PCsTrain"]
PCsVal = data["PCsVal"]
tTrain = data["tTrain"]
tVal = data["tVal"]
std = data["std"]

# Vérifications (à supprimer) ############

print(PCsTrain.shape, PCsVal.shape)
print(tTrain.shape, tVal.shape)


(2922, 150) (730, 150)
(2922,) (730,)


In [2]:
# one-step forecast :

#suppression de la dernière ligne pour l'entrée
X_train_state = PCsTrain[:-1, :]

#suppression de la première ligne de la cible
y_train = PCsTrain[1:, :]

t_train_used = tTrain[:-1]
t_train_used = t_train_used.reshape(-1, 1)

X_train = np.concatenate([X_train_state, t_train_used], axis=1)

# Vérifications (à supprimer) ##############

X_val_state = PCsVal[:-1, :]
t_val_used  = tVal[:-1].reshape(-1, 1)     # (729, 1)
y_val = PCsVal[1:, :]                      # (729, 150)

X_val = np.hstack([X_val_state, t_val_used])  # (729, 151)

print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_val  :", X_val.shape,   "y_val  :", y_val.shape)


X_train: (2921, 151) y_train: (2921, 150)
X_val  : (729, 151) y_val  : (729, 150)


In [3]:
from sklearn.linear_model import Ridge
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np

# Base model
ridge_base = MultiOutputRegressor(Ridge())

pipe_ridge = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", ridge_base)
])

param_grid_ridge = {
    "ridge__estimator__alpha": [0.01, 0.1, 1, 10, 100]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_ridge = GridSearchCV(
    pipe_ridge,
    param_grid_ridge,
    cv=tscv,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

grid_ridge.fit(X_train, y_train)

ridge_best = grid_ridge.best_estimator_

print("Meilleurs hyperparamètres Ridge :", grid_ridge.best_params_)
print("MSE CV Ridge :", -grid_ridge.best_score_)


Meilleurs hyperparamètres Ridge : {'ridge__estimator__alpha': 100}
MSE CV Ridge : 0.718700647354126


In [4]:
y_pred_ridge = ridge_best.predict(X_val)

mse_ridge = mean_squared_error(y_val, y_pred_ridge)
rmse_ridge = np.sqrt(mse_ridge)

print("RMSE Ridge (global) :", rmse_ridge)

rmse_per_mode_ridge = np.sqrt(np.mean((y_val - y_pred_ridge)**2, axis=0))
print("RMSE par mode (Ridge) :", rmse_per_mode_ridge)


RMSE Ridge (global) : 0.9627987274205094
RMSE par mode (Ridge) : [0.39625752 0.50029844 0.5174921  0.57668054 0.6495896  0.61422634
 0.6522364  0.83767    0.7285807  0.8741386  0.7727087  0.8289254
 0.76158124 0.890739   0.7651489  0.799449   0.918801   0.8040187
 0.87714183 0.8766024  0.9098694  0.90245026 0.7986073  0.8484104
 0.8690122  0.8084291  0.88514346 0.87056565 0.8253365  0.9660748
 0.9101814  0.8649687  0.8528794  0.8639875  0.860115   0.92285186
 0.92222506 0.89992183 0.8916467  0.8445249  0.95203614 0.91881645
 0.8929505  0.8362511  0.9514442  0.90347713 0.8628406  0.93183804
 1.0693872  1.0789129  0.9848905  0.9508435  1.1171497  0.9327653
 0.97638875 0.98767585 0.8973125  0.91412103 0.8988933  0.9420458
 1.0054715  0.9544596  0.9834846  0.9516665  0.91317177 0.8798058
 0.96992844 1.093074   1.0505735  1.0270013  1.0243638  0.9065121
 1.0274105  0.9401182  0.9857165  0.9874809  0.99427944 1.016098
 0.9030879  1.089341   0.9383964  0.9743601  0.8944418  1.0462297
 0.97895

In [5]:
import numpy as np

def make_sequences(PCs, window):
    X_seq = []
    y_seq = []
    for i in range(len(PCs) - window):
        X_seq.append(PCs[i:i+window, :])      # (window, 150)
        y_seq.append(PCs[i+window, :])        # (150,)
    return np.array(X_seq), np.array(y_seq)

window = 10  # longueur de séquence

X_seq_train, y_seq_train = make_sequences(PCsTrain, window)
X_seq_val,   y_seq_val   = make_sequences(PCsVal, window)

print("X_seq_train:", X_seq_train.shape)  # (n_samples, window, 150)
print("y_seq_train:", y_seq_train.shape)  # (n_samples, 150)


X_seq_train: (2912, 10, 150)
y_seq_train: (2912, 150)


In [6]:
from tensorflow import keras
from tensorflow.keras import layers
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np

n_timesteps = X_seq_train.shape[1]   # = window
n_features  = X_seq_train.shape[2]   # = 150

def build_lstm_model(n_units=64, learning_rate=1e-3):
    inputs = keras.Input(shape=(n_timesteps, n_features))
    x = layers.LSTM(n_units)(inputs)
    outputs = layers.Dense(n_features)(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="mse"
    )
    return model

lstm_reg = KerasRegressor(
    model=build_lstm_model,
    n_units=64,
    learning_rate=1e-3,
    epochs=50,
    batch_size=32,
    verbose=0,
)


In [7]:
param_grid_lstm = {
    "n_units": [32, 64],
    "learning_rate": [1e-3, 5e-4],
    "batch_size": [32],
    "epochs": [50],
}

grid_lstm = GridSearchCV(
    lstm_reg,
    param_grid_lstm,
    cv=3,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

grid_lstm.fit(X_seq_train, y_seq_train)

print("Meilleurs hyperparamètres LSTM :", grid_lstm.best_params_)

lstm_best = grid_lstm.best_estimator_

y_pred_lstm = lstm_best.predict(X_seq_val)
mse_lstm = mean_squared_error(y_seq_val, y_pred_lstm)
rmse_lstm = np.sqrt(mse_lstm)
print("RMSE LSTM (global) :", rmse_lstm)


Meilleurs hyperparamètres LSTM : {'batch_size': 32, 'epochs': 50, 'learning_rate': 0.0005, 'n_units': 32}
RMSE LSTM (global) : 1.2018203597559927


In [8]:
from tensorflow.keras.layers import GRU

def build_gru_model(n_units=64, learning_rate=1e-3):
    inputs = keras.Input(shape=(n_timesteps, n_features))
    x = GRU(n_units)(inputs)
    outputs = layers.Dense(n_features)(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="mse"
    )
    return model

gru_reg = KerasRegressor(
    model=build_gru_model,
    n_units=64,
    learning_rate=1e-3,
    epochs=50,
    batch_size=32,
    verbose=0,
)

param_grid_gru = {
    "n_units": [32, 64],
    "learning_rate": [1e-3, 5e-4],
    "batch_size": [32],
    "epochs": [50],
}

grid_gru = GridSearchCV(
    gru_reg,
    param_grid_gru,
    cv=3,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

grid_gru.fit(X_seq_train, y_seq_train)

print("Meilleurs hyperparamètres GRU :", grid_gru.best_params_)

gru_best = grid_gru.best_estimator_

y_pred_gru = gru_best.predict(X_seq_val)
mse_gru = mean_squared_error(y_seq_val, y_pred_gru)
rmse_gru = np.sqrt(mse_gru)
print("RMSE GRU (global) :", rmse_gru)


Meilleurs hyperparamètres GRU : {'batch_size': 32, 'epochs': 50, 'learning_rate': 0.0005, 'n_units': 64}
RMSE GRU (global) : 1.1731378490169093
